In [ ]:
from pathlib import Path
import polars as pl
from dtale import show
import dtale.global_state as global_state

global_state.set_app_settings(dict(max_column_width=300))

data_dir = Path().absolute() / ".." / "data"

In [ ]:
df = pl.read_parquet(data_dir / "dagster/parsed_whatsapp_conversations/cm0i27jdj0000aqpa73ghpcxf.snappy")
show(df.to_pandas()).open_browser()


In [ ]:

messages_df = df.filter(pl.col("from").eq("Estela") | pl.col("to").eq("Estela")).with_columns(
    msg=pl.concat_str(
        [
            pl.lit("From: "),
            pl.col("from"),
            pl.lit(", To: "),
            pl.col("to"),
            pl.lit(", Date: "),
            pl.col("datetime").dt.strftime("%Y-%m-%d %H:%M:%S"),
            pl.lit(", Content: "),
            pl.col("content"),
        ]
    ).alias("msg"),
    date=pl.col("datetime").dt.date()
)

# Group and iterate by date
for date in messages_df.get_column("date").unique().sort():
    print(f"\n=== Messages from {date} ===\n")
    
    daily_messages = messages_df.filter(pl.col("date") == date).get_column("msg").to_list()
    for msg in daily_messages:
        print(msg)
    
    print("-" * 100)
